# Iniciando um novo database
E inserir documentos manualmente para fins didáticos do uso de insert, update, delete.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# instalação do mongodb
!apt install mongodb >log
!service mongodb start



 * Starting database mongodb
   ...done.


In [3]:
from pprint import pprint

In [4]:
import pymongo
client = pymongo.MongoClient()

# database 'iotdb' vai ser criado:
db = client.get_database('iotdb')

# Criar 3 dispositivos
A seguir criamos 3 devices manualmente usando dicionários no python, que serão convertidos para o formato json no mongodb.

Cada device representa um documento no mongodb.



In [5]:
# Definir device 1:
dev1 =  {
  '_id': 1,
  'model': 'arduino',
  'latlng': (-25.593778,-54.490199),
  'sensors': ['temperature', 'humidity']
}
pprint(dev1)

{'_id': 1,
 'latlng': (-25.593778, -54.490199),
 'model': 'arduino',
 'sensors': ['temperature', 'humidity']}


In [6]:
dev2 = {
  '_id': 2,
  'model': 'raspberry',
  'latlng': (-25.504422,-54.575832),
  'sensors': ['temperature', 'pressure', 'light']
}
pprint(dev2)

{'_id': 2,
 'latlng': (-25.504422, -54.575832),
 'model': 'raspberry',
 'sensors': ['temperature', 'pressure', 'light']}


In [7]:
dev3 = {
  '_id': 3,
  'model': 'raspberry',
  'latlng': (-25.509473,-54.596119),
  'sensors': ['temperature', 'sound', 'vibration']
}
pprint(dev3)

{'_id': 3,
 'latlng': (-25.509473, -54.596119),
 'model': 'raspberry',
 'sensors': ['temperature', 'sound', 'vibration']}


In [8]:
# SE precisar recriar a collection devices, basta fazer um drop na collection:
# db.devices.drop()

In [9]:
# Aqui efetivamente salvamos cada device na collection devices

db.devices.insert_one(dev1)

db.devices.insert_one(dev2)

db.devices.insert_one(dev3)

In [10]:
list(db.devices.find())

[{'_id': 1,
  'latlng': [-25.593778, -54.490199],
  'model': 'arduino',
  'sensors': ['temperature', 'humidity']},
 {'_id': 2,
  'latlng': [-25.504422, -54.575832],
  'model': 'raspberry',
  'sensors': ['temperature', 'pressure', 'light']},
 {'_id': 3,
  'latlng': [-25.509473, -54.596119],
  'model': 'raspberry',
  'sensors': ['temperature', 'sound', 'vibration']}]

In [11]:
# criar um atributo data que contém um vetor com as medições
dev1['data'] = [
  {'ts': 1639802762, 'temp': 33.2, 'hum': 12.1},
  {'ts': 1639802822, 'temp': 31.1, 'hum': 10.1},
  {'ts': 1639802884, 'temp': 32.2, 'hum': 15.1}
]

In [12]:
import pprint
pprint.pprint(dev1)

{'_id': 1,
 'data': [{'hum': 12.1, 'temp': 33.2, 'ts': 1639802762},
          {'hum': 10.1, 'temp': 31.1, 'ts': 1639802822},
          {'hum': 15.1, 'temp': 32.2, 'ts': 1639802884}],
 'latlng': (-25.593778, -54.490199),
 'model': 'arduino',
 'sensors': ['temperature', 'humidity']}


In [13]:
# UPDATE opção 1:
# vamos atualizar o device device(id=1) no mongodb setando apenas o campo 'data'
#db.devices.update_one({'_id': 1}, {"$set": { 'data': dev1['data'] } } )

In [14]:
# UPDATE opção 2:
# também podemos fazer o update (substituição) do documento inteiro:
db.devices.replace_one({'_id': 1}, dev1)

In [15]:
# Como já vimos o mongodb é schema-less, permitindo que dev1 tenha o atributo 
# data enquanto dev2 e dev3 não possuem dados nesse exemplo
locations = []
for dev in db.devices.find():
  pprint.pprint(dev)
  locations.append((dev['latlng'][0],dev['latlng'][1]))

{'_id': 1,
 'data': [{'hum': 12.1, 'temp': 33.2, 'ts': 1639802762},
          {'hum': 10.1, 'temp': 31.1, 'ts': 1639802822},
          {'hum': 15.1, 'temp': 32.2, 'ts': 1639802884}],
 'latlng': [-25.593778, -54.490199],
 'model': 'arduino',
 'sensors': ['temperature', 'humidity']}
{'_id': 2,
 'latlng': [-25.504422, -54.575832],
 'model': 'raspberry',
 'sensors': ['temperature', 'pressure', 'light']}
{'_id': 3,
 'latlng': [-25.509473, -54.596119],
 'model': 'raspberry',
 'sensors': ['temperature', 'sound', 'vibration']}


In [16]:
# extraindo lat long num array para usar no gmaps
locs = [(lt[0],lt[1]) for lt in locations]
locs

[(-25.593778, -54.490199), (-25.504422, -54.575832), (-25.509473, -54.596119)]

In [18]:
from google.colab import output
output.enable_custom_widget_manager()

In [21]:
# instalar o gmaps
!pip install gmaps

     |████████████████████████████████| 1.1 MB 5.2 MB/s 
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076111 sha256=97ac65fc9f6636036a880b091382f6a1da6870096eec437c79c0cb2edffbf631
  Stored in directory: /root/.cache/pip/wheels/3d/1c/cb/c46f4199210e632b28d78d05a00bfdccbf6dfcd3f5b051d6bf
Successfully built gmaps


In [17]:
# Questão:

# Identifique o nome dos locais onde os devices estão localizados plotando no mapa as coordenadas

In [22]:
# VER exemplos: https://github.com/pbugnion/gmaps
import gmaps
import gmaps.datasets

# Obter api_key: https://console.cloud.google.com/projectselector2/google/maps-apis/credentials

gmaps.configure(api_key="GERAR_A_SUA_API_KEY") # Your Google API key

fig = gmaps.figure()

fig.add_layer(gmaps.marker_layer(locs))

fig

Figure(layout=FigureLayout(height='420px'))